### OK

In [ ]:
### Imports ###
import numpy as np
import string
import math
from scipy.integrate import odeint
import matplotlib.pyplot as pypl
import control   #pip install control, conda install -c python-control -c cyclus slycot control

A=np.array([[0],[0]])
B=np.array([[14],[0]])
tend=18
t=np.arange(0,tend+0.005,0.005)

### OK

In [ ]:
# ------------------------------------------------------------------------ #  
# ------------------------------------------------------------------------ #
# ------------------------------------------------------------------------ #
# ------------------------------------------------------------------------ #
#####
### CONTROLES calculs de v(t) et u(t)
#####
    
def v(t):
    out = 2 + (t>5)*(t<10)
    return out

def u(t):
    out = -0.22*t+(0.44*t)*(t>5)-(0.5*(t-5))*(t>9)+(0.5*(t-13))*(t>13)+(0.2*(t-8))*(t>15) 
    return out

### OK

In [ ]:
#####
### CALCUL DE RHO_2
#####


def AK(z1,z2,z3,B):
# Determination de rho2 en fonction de la distance AB, de l'angle xsi et de
# rho1
#
# formule obtenue ? patir de la formule d'Al-Kashi, valable uniquement
# lorsque x \in [xA;xB]
#
    radicande = (B[0]**2-(z2*math.sin(z3-z1))**2)*(B[0]**2-(z2*math.sin(z3-z1))**2>0)

    
    out = z2*math.cos(z3-z1)+math.sqrt(radicande)
    return out


###  OK

In [ ]:
#####
### FONCTION SymRechape
#####
def SymReshape(u):
    # cas 1 --> transforme une matrice carré symétrique en vecteur de taille n(n+1)/2
    #ou
    # cas 2 --> transforme un vecteur de taille n(n+1)/2 en matrice carré symétrique
    global vector,matrix
    if(len(u.shape)==1): #si vecteur ligne
        m=1
        n=len(u)
    else:
        [m,n]=u.shape
    vector=[]
    matrix=[]
    if (m==n):
        #si cas 1
        print("matrix to vector")
        #if (m==1 and n==1):
            #print("scalaire")
        #else:
            #print ("matrice carre")
        for i in range(0,m):
            for j in range(i,m):
                vector.append(u[i,j])
                    
        vector=np.transpose(np.array(vector))
        return vector
    elif(m==1 or n==1):
        #si cas 2
        print("vector to matrix")
        #if (n==1 and m!=1):
         #   print("vecteur colonne\n")
        if (m==1 and n!=1):
         #   print("vecteur ligne\n")
         #   print(u)
            u=np.transpose(u)
         #   print(u)
            m=n
            n=1
        
        dim=(math.sqrt(1+8*m)-1)/2
        k=0
        if(dim%1==0):
            dim=int(dim)
            matrix=np.zeros((dim,dim))
            for i in range(0,dim):
                for j in range(i,dim):
                    if i==j:
                        matrix[i,j]=u[k]
                    else:
                        matrix[i,j]=u[k]
                        matrix[j,i]=u[k]
                    k+=1
            return matrix
        else:
            print("vecteur pas de la bonne dimention!!")
    else:
         print("ce n est ni une matrice carree, ni un vecteur!!")


### OK

In [ ]:
#
#
#####
### MODELE SOUS FORME NORMALE
#####
#
#


def A_mat(t):
    out = np.array([[0,           0,          0], 
                    [0,           0,          0],
                    [0,           0,          0]])
    return out

def b_mat(t,z):
    global B
   
    ak = AK(z[0],z[1],z[2],B)   
    
    out=np.array([[v(t)*math.sin(z[0])/z[1]-u(t)],
                 [-v(t)*math.cos(z[0])],
                [v(t)*math.sin(z[2])/ak-u(t)]])

    return out


### OK

In [ ]:
def Db_mat(t,z):

    global B

    xb=B[1]
    xi=z[2]-z[0]

    den = AK(z[0],z[1],z[2],B)

    AA=-v(t)*math.sin(z[2])*(z[1]*math.sin(xi)+z[1]**2*math.sin(xi)*math.cos(xi)/math.sqrt(abs(xb**2-(z[1]*math.sin(xi))**2)))/den**2

    BB=-v(t)*math.sin(z[2])*(math.cos(xi)-z[1]*math.sin(xi)**2/math.sqrt(abs(xb**2-(z[1]*math.sin(xi))**2)))/den**2
    
    C1=v(t)*math.cos(z[2])/den 
    C2=v(t)*math.sin(z[2])*(z[1]*math.sin(xi)+z[1]**2*math.cos(xi)*math.sin(xi)/math.sqrt(abs(xb**2-(z[1]*math.sin(xi))**2)))/den**2
    
    CC=C1+C2

    
    out= np.array([[v(t)*math.cos(z[0])/z[1],   -v(t)*math.sin(z[0])/(z[1]**2),    0],
                  [v(t)*math.sin(z[0]),                   0,                       0],
                  [ AA,                                   BB,                    CC ]])

    return out


### OK

In [ ]:
#####
### KALMAN ETAPE DE PREDICTION
#####
def Kalman_Pred_Step(t,X,theta, Qtheta):
    # Prediction step of the continuous-discrete asynchronous K filter   
    ### Retrieve state / estimation / Riccati equation 
    n = 3
    nRic = n*(n+1)/2
    [m,mm]=X.shape
    if(m==1 and mm!=1):
            X=np.transpose(X)
    if (nRic%1 ==0):
        nRic=int(nRic)
    else:
        print("probleme de dimention")
    Xhat=[]
    X2=[]
    # --- #                
    for i in range(0,n):
        Xhat.append(X[i,0])
    
    for i in range(n,n+nRic):
        X2.append(X[i,0])
        
    X2=np.array([X2])
    S = SymReshape(X2)
    # ----- #    
    # No Output related Data
    # ----- #
    #####convert to matrix#####
    b_m=b_mat(t,Xhat)
    Xhat=np.matrix(Xhat)
    Xhat=np.transpose(Xhat)
    A_m=np.matrix(A_mat(t))
    Db_m=np.matrix(Db_mat(t,Xhat))
    S=np.matrix(S)
    Qtheta=np.matrix(Qtheta)
    
    dXhat = np.array(A_m* Xhat + b_m) #-P*C'*IR_t*C*(Xhat-mes(:,where));                    #etat estime 
    # --- # 
    
    dS =-np.transpose(A_m+Db_m)*S-S*np.transpose(A_m+Db_m)-S*Qtheta*S #matrice de Riccati
    # --- #

    SR=SymReshape(dS)
    RS=np.reshape(SR,(6,1))
    
    out = np.concatenate((dXhat,RS),axis=0)
    return out

##### odeset verifier si on en a encore besoin, ligne 180 à 217, plot

In [ ]:
#def Kalman_asynchrone_NB():
import numpy as np
import string
#from math import *
import math
from scipy.integrate import odeint
import matplotlib.pyplot as pypl


global B

###      ###
# Couleurs #
###      ###
C_trajectory = np.array([0, 0, 0.5]) # Bleu Navy
C_rotating_Part = np.array([0, 100/255, 0]) # Vert Sombre
C_Boat2Amer = np.array([176/255, 196/255,  222/255]) # bleu acier clair
C_amer1 = np.array([255/255,20/255,147/255]) # Rose intense
C_amer2 = np.array([210/255,105/255,30/255]) # Chocolat

###        ###
# Parametres #
###        ###
#
#si Display=0: la simulation de la trajectoire du bateau n'apparait pas. 
#Si Display=1: Affichage de la trajectoire
Display=False



###
#  Options de ode
#   options = odeset('RelTol',1e-10,'AbsTol',1e-10,'MaxStep',1e-2)

A=np.array([[0],[0]])     # balise A1
B=np.array([[14],[0]])   # balise A2
tend = 18
t=np.arange(0,tend+0.005,0.005)


###                              ###
# X=(x,y,theta) et \dot{X}=AX+B(u) #
###         
    
    ###
    
def f1(x0,t,u,v):
    u0=u(t)
    v0=v(t)
    x1, x2, x3=x0
    dfdt=[v0*np.cos(x3),v0*np.sin(x3), u0]
    return dfdt   #f(y,t)=y 

def TheBoat(x,y,theta):
    Xboat =np.array([ -0.3750,    0.3750,    0.7000,    0.3750,   -0.3750,  -0.3750])
    Yboat = np.array([0.1,  0.1,  0,  -0.1,  -0.1,  0.1])
 
    Rotation = np.array([[  np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta) ]]) 
    out = Rotation.dot(np.array([Xboat,Yboat])) +  np.array([[x],[y]])*(np.ones(6))
                                              
    return out

#t = np.linspace(0,18,1000)
x0=np.array([1,6,1])

XOUT = odeint(f1,x0,t,args=(u,v))


    #XOUT(:,1) correspond a x(t), XOUT(:,2) correspond a y(t),XOUT(:,3) correspond a theta(t)

#XOUT[:,2]= math.fmod(XOUT[:,2],(2*math.pi));
for i in range(0,len(t)):
    XOUT[i,2]=(XOUT[i,2]%(2*math.pi))
    #print('i=',i,', y[i]=',y1[i],', x[i]=',x[i])
    

ya=np.zeros(len(t))
yb=np.zeros(len(t))
    
for i in range(0,len(t)):
        #mesure de l'angle entre l'oriantation du bateau et la balise A
    ya[i]=(math.atan2((A[1]-XOUT[i,1]),(A[0]-XOUT[i,0]))-XOUT[i,2])%(2*math.pi);

        #mesure de l'angle entre l'oriantation du bateau et la balise B
    yb[i]= (math.atan2((B[1]-XOUT[i,1]),(B[0]-XOUT[i,0]))-XOUT[i,2])%(2*math.pi);
    
print(ya)
print(yb)
    
    
##### ------------------------------------------------------  #####
# Computation Asynchronous mesurements
#
#
#
#  
w=10 #vitesse de rotation du capteur angulaire 
Angle_temp=np.zeros(len(t))
for i in range(0,len(t)):
    Angle_temp[i]=(w*t[i])%(2*math.pi)      #  Angle_temp(:,1)=mod(w*t(1,:),2*pi);

print("ya",ya,"\n yb",yb)
print("angle temp:",Angle_temp, len(Angle_temp))
S1=[]
S2=[]
ii=[]
for i in range (0,len(t)-1):
    if (abs(ya[i]-Angle_temp[i])<1e-1):
        S1.append( t[i])
        ii.append(i)
        #print('OK')
    if (abs(yb[i]-Angle_temp[i])<1e-1):
        S2.append( t[i])
        ii.append(i)
        #print('OK')


print('S1', len(S1))
print('S2', len(S2))
print('max(ya)',max(ya),'\n','min(ya)',min(ya),'\n','max(yb)',max(yb),'\n','min(yb)',min(yb),'\n','moyenne ya', np.mean(ya),'\n','moyenne yb', np.mean(yb))

####   Next - Remove Multi-instances #####
        
#for k in range(0,len(S1)-1):
#    if (S1[k+1]-S1[k]<1e-1):
#        S1[k+1]=(S1[k]+S1[k+1])/2
#        S1[k]=0
#print('S1=',S1)




#fonction qui fait la moyenne des elements d une liste dont la difference 
#est inferieur a epsilon et les revoie sous forme de liste
def rm_multi_instance(S1,epsilon):     
    S2 = [] 
    S3 = []
    i=0
    
    while i<len(S1):
        S2=[] 
        S2.append(S1[i])

        if i+1<len(S1):

            while abs(S1[i+1]-S1[i]) <= epsilon:

                S2.append(S1[i+1])
                i+=1

                if i>=(len(S1)-1):
                    break

        S3.append(np.mean(S2))

        i+=1
        
    return S3    

print('\n\nS1 avant remove',S1, '\n\nlen S1', len(S1))

S1=(rm_multi_instance(S1,1e-1)).copy()
print('\n\nS1 apres remove',S1, '\n\nlen S1', len(S1), '\n\n')


print('\n\nS2 avant remove',S2, '\n\nlen S2', len(S2))

S2=(rm_multi_instance(S2,1e-1)).copy()
print('\n\nS2 apres remove',S2, '\n\nlen S2', len(S2), '\n\n')





#temp1=np.where(S1 != 0)#[0]                         ## temp1=find(S1);
temp1=[]
temp2=[]
for i in range(0,len(S1)):
    if (S1[i]!=0):
        temp1.append(i)
        temp2.append(S1[i])


#temp2=np.zeros(len(S1))
print('taille de temp1:',len(temp1),'\n', 'temp1:', temp1, '\n', 'taille de temp2:', len(temp2),'\n', 'temp2:',temp2)


#for i in range(0,len(temp1)):
#    temp2[i]=S1[temp1[i]];
#S1=np.zeros(len(temp2));
#S1=temp2;
 
for k in range(0,len(S2)-1):
    if (S2[k+1]-S2[k]<1e-1):
        S2[k+1]=(S2[k]+S2[k+1])/2
        S2[k]=0
        #print('test2')


#for i in range(0,len(temp1)):
    #temp2[i]=S1[temp1[i]];
#S1=np.zeros(len(temp2));
#S1=temp2;

#temp2=np.where(S2 == 0)[0]  
#for i in range(0,len(temp1)-1):
#    temp2[i]=S2[temp1[i]]
    #print('test3')

S2=np.zeros(len(temp2))
S2=temp2.copy()
print('S1', len(S1))
print('S2', len(S2))


#         END % Computation Asynchronous mesurements              #
##### ------------------------------------------------------  #####
 
 
 
##### ------------------------------------------------------ #####
# si <Display = 1>
# Pour tracer la trajectoire du bateau "en temps reel+ le capteur qui
# tourne+la droite qui relie le bateau -> la balise.
 
if Display: 
    FigHandle=pypl.figure(1)       #,figsize=(8,6), dpi=80)
    for i in range(0,len(t)):
        pypl.clf
        #hold on
        #pypl.gca().set_aspect('equal', adjustable='box')        #axis('square')
        pypl.xlim(-2, 15)
        pypl.ylim(-1, 13)                                         #axis([-2,12,-5,12])
        ###      pypl.plot([A[0], B[0]],[A[1], B[1]],'--','Color','k')
        pypl.plot(A[0],A[1],marker='o')#,color='C_amer1')
        pypl.plot(B[0],B[1],marker='o')#,color='C_amer2')
        #pypl.plot(A[0,0], A[1,0],'o','Color',C_amer1)
        #pypl.plot(B[0,0], B[1,0],'o','Color',C_amer2)
 
        pypl.plot(XOUT[:,0], XOUT[:,1],linewidth='0.2')#,color='C_trajectory')
        pypl.plot(XOUT[i,0], XOUT[i,1],'or',linewidth='3')
        B_graph = TheBoat(XOUT[i,0],XOUT[i,1],XOUT[i,2]);
        pypl.plot(B_graph[0,:],B_graph[1,:],'k',LineWidth='1.5')
 
        #  Face a l'amer.        
        #  if or((abs(ya(i,1)-Angle_temp(1,i))<1e-2),(abs(yb(i,1)-Angle_temp(1,i))<1e-2))
 
        if (math.fabs(ya[i]-Angle_temp[i])<1e-1):
            pypl.plot([XOUT[i,0],XOUT[i,0]+5*np.cos(XOUT[i,2]+Angle_temp[i])], [XOUT[i,1],XOUT[i,1]+5*np.sin(XOUT[i,2]+Angle_temp[i])], 'b')#,color='C_amer1',lineStyle=':',lineWidth='2')
        elif (math.fabs(yb[i]-Angle_temp[i])<1e-1):
            pypl.plot([XOUT[i,0],XOUT[i,0]+5*np.cos(XOUT[i,2]+Angle_temp[i])], [XOUT[i,1],XOUT[i,1]+5*np.sin(XOUT[i,2]+Angle_temp[i])],'g')#,color='C_amer2',lineStyle=':',lineWidth='2')
        else:
            pypl.plot([XOUT[i,0],XOUT[i,0]+5*np.cos(XOUT[i,2]+w*t[i])], [XOUT[i,1],XOUT[i,1]+5*np.sin(XOUT[i,2]+w*t[i])],'r')#,color='C_rotating_Part', lineStyle=':',lineWidth='2')
          
 
        #pypl.plot([XOUT[i,0],A[0]], [XOUT[i,1],A[1]])#,linewidth='1',color='C_Boat2Amer')    # droites bateau - amer
 
        #pypl.plot([XOUT[i,0],XOUT[i,0]+np.cos(XOUT[i,2])], [XOUT[i,1],XOUT[i,1]+np.sin(XOUT[i,2])])#,'k',linewidth='2')
 
        #pypl.plot([XOUT[i,0],XOUT[i,0]+np.cos(XOUT[i,2]+ya[i])],[XOUT[i,1],XOUT[i,1]+np.sin(XOUT[i,2]+ya[i])])#,color='C_amer1', linewidth='2')
        #pypl.plot([XOUT[i,0],XOUT[i,0]+np.cos(XOUT[i,2]+yb[i])],[XOUT[i,1],XOUT[i,1]+np.sin(XOUT[i,2]+yb[i])])#,color='C_amer2', linewidth='2')
 
        #pypl.plot([XOUT[i,0],B[0]], [XOUT[i,1],B[1]])#,linewidth='1',color='C_Boat2Amer')
        #pypl.draw()
        pypl.show()
    
#print(temp2)    
    

##### plot

In [ ]:
###########
#### Passage en coordonnees polaires:
#### (x,y,theta)--> (alpha1, alpha2, rho1,rho2) 

alpha1=np.zeros((len(t),1))
alpha2=np.zeros((len(t),1))
rho1=np.zeros((len(t),1))
rho2=np.zeros((len(t),1))
x1=np.zeros((len(t),1))
x2=np.zeros((len(t),1))
y1=np.zeros((len(t),1))
y2=np.zeros((len(t),1))


for i in range (len(t)):
    alpha1[i]=math.atan2(XOUT[i,1]-A[1],XOUT[i,0]-A[0])#-math.pi
    rho1[i]  =math.sqrt((XOUT[i,1]-A[1])**2+(XOUT[i,0]-A[0])**2)
    alpha2[i]=math.atan2(XOUT[i,1]-B[1],XOUT[i,0]-B[0])#-math.pi
    rho2[i]  =math.sqrt((XOUT[i,1]-B[1])**2+(XOUT[i,0]-B[0])**2)

     
    
for i in range (len(t)):
    x1[i]=rho1[i]*math.cos(alpha1[i])
    y1[i]=rho1[i]*math.sin(alpha1[i])
    x2[i]=rho2[i]*math.cos(alpha2[i])+B[0]
    y2[i]=rho2[i]*math.sin(alpha2[i])+B[1]
   

"""
    A convertir en Python

    #     hold on
    #     plot(alpha2(1,i)*cos(rho2(1,i)), alpha2(1,i)*sin(rho2(1,i)))
    #     drawnow

    
    
    figure(2)
    #axis('square')
    axis([-2,12,-5,12])
    hold on
    plot([A(1,1), B(1,1)],[A(2,1), B(2,1)],'--','Color','k')
    plot(A(1,1), A(2,1),'o','Color',C_amer1)
    plot(B(1,1), B(2,1),'o','Color',C_amer2)

    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory,'LineWidth',2);
    plot(x1(:,1),y1(:,1),'Color','r','LineWidth',1);
    plot(x2(:,1),y2(:,1),'Color','y','LineWidth',0.5);
    legend('','','','Trajectoire -coor. originales','trajectoire -- coor. poliares w.r.t A1','trajectoire -- coor. poliares w.r.t A2')

    hold off
    ###
    ### FIN FIGURE
    #########
    
"""

##### ode 45 fait , plot

In [ ]:
#####                         #####
### DYNAMIQUE EN COORD POLAIRES ###
#
#
#
    
# Pol_A_0 = coordonnee initiale en representation polaire w.r.t. A
Pol_A_0 = [math.sqrt((XOUT[0,1]-A[1])**2+(XOUT[0,0]-A[0])**2),
            math.atan2(XOUT[0,1]-A[1],XOUT[0,0]-A[0]),                 
            XOUT[0,2]]               #pourquoi pas alpha1=math.atan2(XOUT[0,1]-A[1],XOUT[0,0]-A[0])-math.pi??

print('Pol_A_0',Pol_A_0)


# x1 = rho1, x2 = alpha1, x3 = theta        

def g1(x0,t,u,v):
    u0=u(t)
    v0=v(t)
    x1, x2, x3=x0
    g1=[v0*np.cos(x3-x2),v0*np.sin(x3-x2)/x1, u0]
    return g1   #f(y,t)=y 
                                      
POL1OUT = odeint(g1,Pol_A_0,t,args=(u,v))                                      
print('POL1OUT',POL1OUT[0],POL1OUT[10],POL1OUT[100])                                        
                                                                      
                                                             


#Pol_B_0 = coordonnee initiale en representation polaire w.r.t. A
Pol_B_0 = [math.sqrt((XOUT[0,1]-B[1])**2+(XOUT[0,0]-B[0])**2),
            math.atan2(XOUT[0,1]-B[1],XOUT[0,0]-B[0]),    #idem
            XOUT[0,2]]             #pourquoi pas alpha2=math.atan2(XOUT[0,1]-B[1],XOUT[0,0]-B[0])-math.pi ??


print('Pol_B_0',Pol_B_0)

# x1 = rho2, x2 = alpha2, x3 theta        

                                      
POL2OUT = odeint(g1,Pol_B_0,t,args=(u,v))  
print('POL2OUT',POL2OUT[0],POL2OUT[10],POL2OUT[100])
                                   



"""
A convertir

    figure(3)
    title('trajectoire donnee par la dynamique en coord. polaires')
    hold on
%    axis([-2,30,-5,12])
    plot([A(1,1), B(1,1)],[A(2,1), B(2,1)],'--','Color','k')
    plot(A(1,1), A(2,1),'o','Color',C_amer1)
    plot(B(1,1), B(2,1),'o','Color',C_amer2)
    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory,'LineWidth',3);

    plot(POL1OUT(:,1).*cos(POL1OUT(:,2)),POL1OUT(:,1).*sin(POL1OUT(:,2)),'r','LineWidth',1.5)

    plot(POL2OUT(:,1).*cos(POL2OUT(:,2))+B(1,1),POL2OUT(:,1).*sin(POL2OUT(:,2))+B(2,1),'g','LineWidth',0.5)

    hold off

    Measure1_polA = mod(POL1OUT(:,2)-POL1OUT(:,3)+pi,2*pi); % Xi_1 = alpha1 + pi -Theta

                                      %
                                      %
                                      % 
    % % DYNAMIQUE EN COORD POLAIRES % %
    %%%%%       -------------     %%%%%

                     
"""            

                 

##### code sur les diff a convert pour les mesures ( à verifier) , Noisy à verifier, plot

In [ ]:
####           #####
# CALCUL DES SORTIES
#
#
# 



theta = XOUT[:,2]
mes_1=[]
mes_3=[]

print('theta',theta)
print('alpha1',alpha1)
print('alpha2',alpha2)


mes_2=[]
for i in range (len(rho1)):
    mes_2.append(rho1[i,0])

for i in range (len(theta)):
    mes_1.append( math.pi - theta[i]+alpha1[i,0] )
    mes_3.append( math.pi - theta[i]+alpha2[i,0] )
    
   
mes_1_V2 = mes_1.copy() 
print("\n\n\n\n\n\n\nmes_1_V2",mes_1_V2)


indexes=[]
        
for i in range (0,len(mes_1)-1):
    if (mes_1[i+1]-mes_1[i])<-0.5:       
        indexes.append(i)
        print(indexes)

mes_1_V3=[]        
if len(indexes)>0:
    for j in range(len(indexes)):
        for i in range(len(indexes)):
            mes_1_V3[indexes[i]+1:len(indexes)]=(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)]

indexes=[]

for i in range (0,len(mes_1_V2)-1):
    if (mes_1[i+1]-mes_1[i])<-0.5:       
        indexes.append(i)
        print(indexes)
        
if len(indexes)>0:
    for i in range(len(indexes)):
        mes_1_V2[indexes[i]+1:len(indexes)]=-(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)]



mes_3_V2 = mes_3.copy()
print("\n\n\n\n\n\n\nmes_3_V2",mes_3_V2)




indexes=[]

for i in range (0,len(mes_3)-1):
    if (mes_3[i+1]-mes_3[i])<-0.5:       
        indexes.append(i)
        print(indexes)
        
if len(indexes)>0:
    for i in range(len(indexes)):
        mes_3_V2[indexes[i]+1:len(indexes)]=(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)] 

indexes=[]
for i in range (0,len(mes_3)-1):
    if (mes_3[i+1]-mes_3[i])>0.5:       
        indexes.append(i)
        print(indexes)
        
if len(indexes)>0:
    for i in range(len(indexes)):
        mes_3_V2[indexes[i]+1:len(indexes)]=-(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)]     
        
        

   
#code a convertir code initial

####                            ####
# Mesures d'angle sans modulo 2*pi #
#mes_1_V2 = mes_1.copy() 
#indexes = find(diff(mes_1)<-0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_1_V2[indexes(i)+1:end]=(2*math.pi)+mes_1_V2[indexes(i)+1:end]
    
    
#indexes = find(diff(mes_1_V2)>0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_1_V2[indexes(i)+1:end]=-(2*math.pi)+mes_1_V2[indexes(i)+1:end]
    
    
#mes_3_V2 = mes_3.copy()
#indexes = find(diff(mes_3)<-0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_3_V2[indexes(i)+1:end)]=(2*math.pi)+mes_3_V2[indexes(i)+1:end]
        
    
#indexes = find(diff(mes_3_V2)>0.5)
#if len(indexes)>0:
#    for i in range len(indexes):
#        mes_3_V2[indexes(i)+1:end]=-(2*math.pi)+mes_3_V2[indexes(i)+1:end]


#Verifier dans la conversion si transposée nécessaire
#Measures = np.array([[mes_1_V2],
#                     [mes_2],
#                     [mes_3_V2]])    # Format : [mes_1;mes_2;mes_3]




####################
###   Code modifié mais ne fonctionne pas
###################

"""
theta = XOUT[:,2]
mes_1=np.zeros((len(theta),1))
mes_3=np.zeros((len(theta),1))

mes_2=[]
for i in range (len(rho1)):
    mes_2.append(rho1[i,0])


for i in range (len(theta)):
    mes_1[i,0] = math.pi - theta[i]+alpha1[i]
    mes_3[i,0] = math.pi - theta[i]+alpha2[i]

mes_1_V2 = mes_1.copy() 

indexes=[]
        
for i in range (0,len(mes_1)-1):
    if (mes_1[i+1]-mes_1[i])<-0.5:       
        indexes.append(i)
        print(indexes)

if len(indexes)>0:
    for i in range(len(indexes)):
        mes_1_V2[indexes[i]+1:len(indexes)]=(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)]

indexes=[]

for i in range (0,len(mes_1_V2)-1):
    if (mes_1[i+1]-mes_1[i])<-0.5:       
        indexes.append(i)
        print(indexes)
        
if len(indexes)>0:
    for i in range(len(indexes)):
        mes_1_V2[indexes[i]+1:len(indexes)]=-(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)]

        
mes_3_V2 = mes_3.copy(); 
indexes=[]

for i in range (0,len(mes_3)-1):
    if (mes_3[i+1]-mes_3[i])<-0.5:       
        indexes.append(i)
        print(indexes)
        
if len(indexes)>0:
    for i in range(len(indexes)):
        mes_3_V2[indexes[i]+1:len(indexes)]=(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)] 

indexes=[]
for i in range (0,len(mes_3)-1):
    if (mes_3[i+1]-mes_3[i])>0.5:       
        indexes.append(i)
        print(indexes)
        
if len(indexes)>0:
    for i in range(len(indexes)):
        mes_3_V2[indexes[i]+1:len(indexes)]=-(2*math.pi)+mes_1_V2[indexes[i]+1:len(indexes)]     
        
        
print("mes_1_V2 av \n",mes_1_V2)    
#print("mes_2 av \n",mes_2)  
print("mes_3_V2 av \n",mes_3_V2)  

#Verifier dans la conversion si transposée nécessaire
mes_1_V2=np.transpose(mes_1_V2)
#mes_2=np.transpose(mes_2)
mes_3_V2=np.transpose(mes_3_V2)

print("mes_1_V2 ap \n",mes_1_V2)    
#print("mes_2 ap \n",mes_2)  
print("mes_3_V2 ap \n",mes_3_V2)  

Measures = np.array([mes_1_V2,
                     mes_2,
                     mes_3_V2])    # Format : [mes_1;mes_2;mes_3]

"""

Noisy = 1
St_dev1 =0.1
St_dev2 =1 
St_dev3 =0.1


AA = np.array([[0.2], 
               [0.4], 
               [0.3]])
    # 0 : kills the noise
    # 1: keeps the noise
    


#print("Measures",Measures)    
if Noisy: 
    [m,n]=Measures.shape
    Noise =np.random.randn(m,n)
    for i in range (0,3):
        name = "np.std("+str(i)+")"      # a verifier si np.std() est la bonne fonction
        Noise[i,:]=Noise[i,:]*eval(name) 
      
      #Attention aux indices de i !!!!
    for i in range (1,len(Noise)):
        Noise[:,i] = (np.ones((3,1))-AA)*Noise[:,i-1]+AA*Noise[:,i]  #Probleme au niveau des dimensions
        # Noise i <- nouveau signal
        # Noise i-1 <- ancien signal

      
    Measures = Measures + Noise
#    figure(44)
     
      
#    for i in range (0,3):
#        subplot(3,1,i)         ##Verification de indice lors de conversion
#        plot(TOUT,Measures[i,:])

            
                                  #
                                  #
                                  #
######        --------         ####


In [ ]:
 Noise =np.random.randn(4,4)
 print("Noise",Noise)   
print(Noise.shape)
[m,n]=Noise.shape
#Noise[i,:]
Noise2=np.random.randn(m,n)
print("Noise2",Noise2)   

for i in range (0,3):
        name = "np.std("+str(i)+")"
        Noise[i,:]=Noise[i,:]//0.5
        
print("new_Noise",Noise)

AA = np.array([[0.2], 
               [0.4], 
               [0.3]])

print("len(Noise)",len(Noise))
for i in range (1,len(Noise)):
        Noise[:,i] = (np.ones((3,1))-AA)*Noise[:,i-1]+AA*Noise[:,i]
        
print("Noise_two",Noise)

##### ode45 fait , plot

In [ ]:
################             ######
### Forme normale d'observabilite #
#
#
#
zed=np.zeros((len(t),3))
for i in range (len(t)):
    zed[i,0]= math.pi-XOUT[i,2]+alpha1[i,0]
    zed[i,1]= rho1[i]
    zed[i,2]= math.pi-XOUT[i,2]+alpha2[i,0]
    


### ???    phi_exact(:,1)=(sin(z(:,1)).*z(:,4)).^2+(sin(z(:,3)).*z(:,2)).^2-2*sin(z(:,1)).*sin(z(:,3)).*cos(abs(z(:,1)-z(:,3))).*z(:,2).*z(:,4)-(B(1)*z(:,2).*z(:,4)).^2;




### A convertir
###                ### 
# Dynamique Normale  # 
###                ###
#[TOUT,NORM_OUT] = ode45(@(t,z) [v(t)*sin(z(1))/z(2)-u(t);
#                                -v(t)*cos(z(1));
#                                v(t)*sin(z(3))/AK(z(1),z(2),z(3),B)-u(t)],TOUT,zed(1,:),options);

def h1(x0,t,u,v,B):
    u0=u(t)
    v0=v(t)
    x1, x2, x3=x0
    h1=[v0*np.sin(x1)/x2-u0, -v0*np.cos(x1), v0*np.sin(x3)/AK(x1,x2,x3,B)-u0]
    return h1   #f(y,t)=y 
                                      
NORM_OUT = odeint(h1,zed[0,:],t,args=(u,v,B))

print('NORM_OUT',NORM_OUT[0],NORM_OUT[10],NORM_OUT[100])
print(np.shape(NORM_OUT))






#figure(4)
#title('Simulation forme normale VS changement de variables');
# --- #
#subplot(3,1,1)
#plot(TOUT,mod(zed(:,1),2*pi),'k')
#hold on
#plot(TOUT,mod(NORM_OUT(:,1),2*pi),'r')
#hold off
# --- #
#subplot(3,1,2)
#plot(TOUT,zed(:,2),'k')
#hold on
#plot(TOUT,NORM_OUT(:,2),'r')
#hold off
# --- #
#subplot(3,1,3)
#plot(TOUT,mod(zed(:,3),2*pi),'k')
#hold on
#plot(TOUT,mod(NORM_OUT(:,3),2*pi),'r')
#hold off


#    figure(4)
#    plot(phi_exact)
#    title('\phi=\rho_1^2+\rho_2^2-2*\rho_1*\rho_2cos(\xi)-xb avec z exact' )
    ### for i in range (len(t)):
    ###     z[i,0]=alpha1[i,0]-XOUT[i,2]
    ###     z[i,1]=math.sin(XOUT[i,2]-alpha1[i,0])/rho1[i,0]
    ###     z[i,2]=alpha2[i,0]-XOUT[i,2]
    ###     z[i,3]=math.sin(XOUT[i,2]-alpha2[i,0])/rho2[i,0]



#####  plot

In [ ]:
    ### ------------------------------------ ###
    ### changement de variable inverse.
    #
    #
    #
    #
    #
    
    z = NORM_OUT
    
    ### Assurons nous qu'il est possible de reconstituer (x,y,theta) a partir de
    ### (z1, z2, z3,z4)

    ### A un instant t donne, le bateau est sur une courbe de niveau (un arc de
    ### cercle circonstrit au triangle A-B-Bateau. Le centre C de ce cercle est
    ### le point de concours des mediatrices de ce triangle et peut donc etre
    ### calcule a la main. 

    ### En utilisant cela ainsi que le fait que rho1 et rho2 soient les rayons
    ### de cercles centres resp. en A et B et de rayon A-Bateau, resp.
    ### B-Bateau, on peut trouver une expression pour y et donc pour x et
    ### theta. On exploite notamment la connaissance de yA=theta-alpha1,
    ### yB=theta-alpha2 et rho1=sin(z1)/z2 (idem pour rho2)

    #diff(rho1,t)=v*cos(theta-alpha1)=v*cos(z1) (idem pour rho2)

    l=len(t)
    
    xi=np.zeros((l,1))
    C=np.zeros((l,2))
    R=np.zeros((l,1))
    r1=np.zeros((l,1))      #calcul de rho1 a partir de z1 et z2
    r2=np.zeros((l,1))      #calcul de rho1 a partir de z3 et z4
    y=np.zeros((l,1))
    x=np.zeros((l,1))
    a1=np.zeros((l,1))      #calcul de alpha1 et alpha2 a partir de x et y
    a2=np.zeros((l,1))
    tht=np.zeros((l,1))     #calcul de theta a partir de z1 et alpha1 (ou z3 et alpha2)


    Ray=np.zeros((l,1))
    yNUM1=np.zeros((l,1))
    yDEN1=np.zeros((l,1))
    yNUM2=np.zeros((l,1))
    yDEN2=np.zeros((l,1))

    
    erreur_rho=np.zeros((l,1))
    erreur_alpha1=np.zeros((l,1))
    erreur_alpha2=np.zeros((l,1))
    erreur_x=np.zeros((l,1))
    erreur_y=np.zeros((l,1))
    erreur_theta=np.zeros((l,1))
    

    for i in range (l):
        xi[i]=abs(z[i,2]-z[i,0])
        C[i,0]=(B[1]-A[1]+math.tan(xi[i,0])*(A[0]+B[0]))/(2*math.tan(xi[i,0])) #pourquoi C(1) est il constant?
        C[i,1]=(B[0]-A[0]+math.tan(xi[i,0])*(A[1]+B[1]))/(2*math.tan(xi[i,0]))


# Attention verifier adaptation code        
    for i in range (len(C)):
        Ray[i] = math.sqrt( np.dot(C[i],np.transpose(C[i])) )

    #  Ray(:,1)=sqrt((B(1,1)-A(1,1))^2+(B(2,1)-A(2,1))^2)./(2*sin(xi(:,1)));
    #r1(:,1)=sin(pi-z(:,1))./z(:,2);           
    for i in range (len(z)):
        r1[i] = z[i,1]
#    r2(:,1)=sin(pi-z(:,3))./z(:,4);        
    for i in range (l):
        r2[i,0] = AK(z[i,0],z[i,1],z[i,2],B)

    
    for i in range (l):
        yNUM1[i] = (C[i,0]*(r2[i]**2-r1[i]**2)+B[0]*r1[i]**2)

        yDEN1[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        yNUM2[i] = (B[0]*C[i,0]*(C[i,0]-B[0])-B[0]*Ray[i,0]**2+B[0]*C[i,1]**2-C[i,0]*B[1]**2)

        yDEN2[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        
    y =yNUM1/yDEN1 + yNUM2/yDEN2

    
    for i in range (l):
        x[i,0]=math.sqrt(r1[i,0]**2-y[i,0]**2)
        a1[i,0]=(math.atan2(A[1]-y[i,0],A[0]-x[i,0])-math.pi) % (2*math.pi)
        a2[i,0]=(math.atan2(B[1]-y[i,0],B[0]-x[i,0])-math.pi) % (2*math.pi)
        tht[i,0]=math.pi-z[i,0]+a1[i,0] 

        
"""
    figure(5)
    title('Trajectoire reconstruite a partir de la forme normale')
    hold on
    plot(x(:,1),y(:,1),'r')
    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory)
    hold off

"""


    ### Calcul erreur 
    for i in range (l):
        erreur_rho[i,0]=r1[i,0]-rho1[i,0]
        erreur_alpha1[i,0]=alpha1[i,0]-a1[i,0]
        erreur_alpha2[i,0]=alpha2[i,0]-a2[i,0]

        erreur_x[i,0]=XOUT[i,0]-x[i,0]
        erreur_y[i,0]=XOUT[i,1]-y[i,0]
        erreur_theta[i,0]=XOUT[i,2]-tht[i,0]

"""
    figure(6)
    hold on
    title('erreur de reconstruction')
    plot(erreur_alpha2)
    plot(erreur_alpha1)
    plot(erreur_rho )
    plot(erreur_theta)
    #plot(erreur_x)
    #plot(erreur_y)
    hold off
  
"""
                                            
                                               #
                                               #
                                               #
                                               #
    ### changement de variable inverse.        #
    ### ------------------------------------ ###



##### Complet à verifier mais verifier avant si erreur existe en python

In [ ]:
    #####                ---------------------               ####
    #  Asynchronous continuous-discrete High-Gain Kalman Filter #
    #                                                           #
    #####                ---------------------               ####

    
    TimeStamps = np.union1d(S1,S2)
    if TimeStamps[0]==S1[0]:
        FirstIndex = 1
    elif TimeStamps[0]==S2[0]:
        FirstIndex = 2
    else:
        error('Problem With first index')  # Verifier si existe en python

#####  Complet à verifier, verifier que S1, TimeStampsF, S2, TimeStampsS est les meme dimensions, Probleme S1 et S2 pas bonne taille

In [ ]:
    ####                          ###
    # Test d'alternance des indices #
    #
    #
    
    First_indexes=[]
    for i in range (0,len(TimeStamps),2):
        First_indexes.append(i)
    print (First_indexes)
    
    Second_indexes=[]
    for i in range (1,len(TimeStamps),2):    
        Second_indexes.append(i)
    print (Second_indexes)
    
    print(TimeStamps)
    
    TimeStampsF=[]
    for i in range (len(First_indexes)):
        TimeStampsF.append(TimeStamps[First_indexes[i]])
    
    TimeStampsS=[]
    for i in range (len(Second_indexes)):
        TimeStampsS.append(TimeStamps[Second_indexes[i]])
        
    TimeStampsF=np.array(TimeStampsF)
    TimeStampsS=np.array(TimeStampsS)
    
    S1=np.array(S1)
    S2=np.array(S2)
    
    print('\n\n TimeStampsF \n',TimeStampsF,'\n\n TimeStampsS \n',TimeStampsS)
    
    print('\n\n S1 \n ',S1,'\n\n S2 \n',S2)
    
    
    #print(S1==TimeStampsF)
    
    
    if ( ((len(S1)==len(TimeStampsF)) & (len(S2)==len(TimeStampsS))) | ((len(S1)==len(TimeStampsS)) & (len(S2)==len(TimeStampsF))) ):
        if FirstIndex == 1:
            if min(S1==TimeStampsF) & min(S2==TimeStampsS):
                print('Alternance OK')
            else:
                print('Alternance NOT OK')

        else:
            if min(S1==TimeStamps(Second_indexes)) & min(S2==TimeStamps(First_indexes)):
                print('Alternance OK')
            else:
                print('Alternance NOT OK')
    else:
         print('Alternance NOT OK')
    
                                    #
                                    #   
    # Test d'alternance des indices #
    ####                          ###
    

##### Complet à verifier

In [ ]:
    ##### -------------------------
    ### Parametres de l'observateur    
    #
    #
    #
    #
    ### --------------
    # HIGH-GAIN PARAMETER
    #
    #
    HG = 3   # Grand-gain theta
    Delta=np.diag([1,1,1]) # Due to very basic normal form
        
    Q = 1e-2*np.diag([1,10,1])   # ???

#    Q=2e-2*np.eye(3,3)

    Qtheta=HG*np.linalg.inv(Delta)*Q*np.linalg.inv(Delta)
    
    R = np.diag([0.1,1,0.1])
    
#    R = 1e1*np.eye(3,3)
    

    
    
    # -----
    # Initial Guess
 #   xhat0 = np.array([[0.7],[6.3],[1.3]])  # Initial Guess In original coodinates
 #    ya = (math.atan2((A[1]-xhat0[1]),(A[0]-xhat0[0]))-xhat0[2]) % (2*math.pi)

    #mesure de l'angle entre l'orientation du bateau et la balise B
 #   yb = (math.atan2((B[1]-xhat0[1]),(B[0]-xhat0[0]))-xhat0[2]) % (2*math.pi)  
 #    RHO1 = math.sqrt(0.7**2+6.3**2)   
 #    xhat0 =np.array([[ya],[RHO1],[yb]])
      

    xhat0=np.transpose([zed[0]])+np.array([[1],[-1],[-0.5]])   # Initial Guess in Normal coordinates
    
    # Riccati Initial State 
    As = (A_mat(0)+Db_mat(0,xhat0))
#    P0=control.care(np.transpose(As),np.eye((3,3)),Qtheta,R,np.zeros((3,3)),np.eye((3,3))) # cf. cahier de Nico
    
    #S0=np.diag([1,1,1])       
#    S0 = np.linalg.inv(P0)
    
#    S0=SymReshape(S0)    ### A recuperer dans le mail d'Aida
    #
    #
    ###
                                      #
                                      #
    # Parametres de l'observateur     #
    ######### -------------------------    
    
    
    
    

In [ ]:
As1=np.array([[-0.06468883, -1.9727878476642873, -0.03113719],
              [ 0.07636261,                   0,  0.00409228],
              [          0,                   0,  -0.0062017]])
print(As1)
print(np.transpose(As))
print(np.eye(3,3))
print(Qtheta)
print(R)
print(np.zeros((3,3)))
print(np.eye(3,3))
control.care(As1,np.eye(3,3),Qtheta,R,np.zeros((3,3)),np.eye(3,3))

##### ode45 fait mais a verifier, plot, voir "verifier la conversion" (find measure red .... ) et KALOUT2 et Big_TOUT en dessous des ode45

In [ ]:
    sensor = FirstIndex
    Big_TOUT = []
    KALOUT2 = []
    for i in range(len(TimeStamps)):
        if i==0: 
            TSPAN = [0 ,TimeStamps[0]]
#            [TOUT2,KALOUT] = ode45(@(t,Z) Kalman_Pred_Step(t,Z,HG, Qtheta),TSPAN,[xhat0;S0],options); #convertir ode45

                                      
           
#            KALOUT = odeint(Kalman_Pred_Step(t,Z,HG, Qtheta),[xhat0,S0],TSPAN,args=(u,v))  #attention gestion de la fonction et Tout2

            
            
#            KALOUT2 = np.array([[KALOUT2],[KALOUT(1:end,:)]])
#            Big_TOUT = np.array([[Big_TOUT],[TOUT2(1:end,:)]]) 
           
        else:
            TSPAN = [TimeStamps[i-1], TimeStamps[i]]
#             [TOUT2,KALOUT] = ode45(@(t,Z) Kalman_Pred_Step(t,Z,HG, Qtheta),TSPAN,KALOUT2(end,:),options);  #convertir ode45

#            KALOUT = odeint(Kalman_Pred_Step(t,Z,HG, Qtheta),KALOUT2(end,:),TSPAN,args=(u,v))   #attention gestion de la fonction et Tout2



#            Big_TOUT = np.array([[Big_TOUT],[TOUT2(2:end,:)]])      
#            KALOUT2 = np.array([[KALOUT2],[KALOUT(2:end,:)]])       
        
        
        if ((i == 0) or (i == 1)):
            DT = TimeStamps[i]
        else:
            DT = TimeStamps[i]-TimeStamps[i-2]
            
            
        ###             ----------                   ###
        # In order to know which measure is considered #
        #   Where = max(find(TOUT<=TimeStamps[i]))      #        # verifier conversion find
        ###             ----------                   ###
        
        if FirstIndex==1:
           # -- MODEL RELATED -#
            C = np.array([[1,0,0], 
                          [0,1,0]])

#            Red = R(1:2,1:2)                    # verifier conversion
            
            # -- HIGH-GAIN CONSTRUCTION -#  
            
            IR_t=HG*np.linalg.inv(Red)
           
            # -- Measures -- #
#            Output_Signal = Measures(1:2,Where)      # verifier conversion
            
            # ----- #
            FirstIndex= (FirstIndex % 2)+1
        else:    
             # -- MODEL RELATED -#
            C = [0, 0, 1]

#            Red = R(3,3)                              # verifier conversion
            
            # -- HIGH-GAIN CONSTRUCTION -#  
            
            IR_t=HG*np.linalg.inv(Red)

            # -- Measures -- #
#            Output_Signal = Measures(3,Where)         # verifier conversion
            
            # ----- #
            FirstIndex= (FirstIndex % 2)+1
            

        ### Some vectors need to be transposes in order to meet the format
        ### of KALOUT2.
        
#        Zminus = np.transpose(KALOUT2(end,1:3))                     # verifier conversion
#        Sminus = SymReshape(np.transpose(KALOUT2(end,4:end)))       # verifier conversion
        
        Splus = Sminus + np.transpose(C)*IR_t*C*DT
         
        Zplus = Zminus-np.linalg.inv(Splus)*np.transpose(C)*IR_t*(C*Zminus-Output_Signal)*DT
        
#        KALOUT2(end,1:3) = np.transpose(Zplus)                          # verifier conversion
#        KALOUT2(end,4:end) = np.transpose(SymReshape(Splus))            # verifier conversion
        
  

'''
    figure(7) # Convergence dans les coordonnees normales
    
    #title('Simulation forme normale VS changement de variables')
    # --- #
    subplot(3,1,1)
    plot(TOUT,mod(NORM_OUT(:,1),2*pi),'k')
    hold on
    plot(TOUT,mod(zed(:,1),2*pi),'b')
    plot(Big_TOUT,mod(KALOUT2(:,1),2*pi),'r')
    hold off
    legend({'$z_1$','$\hat z_1$'},'Interpreter','latex')
    
    % --- %
    subplot(3,1,2)
    plot(TOUT,NORM_OUT(:,2),'k')
    hold on
    plot(TOUT,zed(:,2),'b')
    plot(Big_TOUT,KALOUT2(:,2),'r')
    hold off
    legend({'$z_2$','$\hat z_2$'},'Interpreter','latex')
    
    % --- %
    subplot(3,1,3)
    plot(TOUT,mod(NORM_OUT(:,3),2*pi),'k')
    hold on
    plot(TOUT,mod(zed(:,3),2*pi),'b')
    plot(Big_TOUT,mod(KALOUT2(:,3),2*pi),'r')
    hold off
    legend({'$z_3$','$\hat z_3$'},'Interpreter','latex')
    
'''      
        
        

##### voir "verifier la conversion", plot

In [ ]:
    ### ------------------------------------ ###
    ### changement de variable inverse (de l'etat estime)
    #
    #
    #
    #
    #
    
    
    for j in range(0,3):
        for i in range(len(KALOUT2)):
            
            z[i,j] = KALOUT2[i,j] 
    
    ### Assurons nous qu'il est possible de reconstituer (x,y,theta) a partir de
    ### (z1, z2, z3,z4)

    ### A un instant t donne, le bateau est sur une courbe de niveau (un arc de
    ### cercle circonstrit au triangle A-B-Bateau. Le centre C de ce cercle est
    ### le point de concours des mediatrices de ce triangle et peut donc etre
    ### calcule a la main. 

    ### En utilisant cela ainsi que le fait que rho1 et rho2 soient les rayons
    ### de cercles centres resp. en A et B et de rayon A-Bateau, resp.
    ### B-Bateau, on peut trouver une expression pour y et donc pour x et
    ### theta. On exploite notamment la connaissance de yA=theta-alpha1,
    ### yB=theta-alpha2 et rho1=sin(z1)/z2 (idem pour rho2)

    #diff(rho1,t)=v*cos(theta-alpha1)=v*cos(z1) (idem pour rho2)    # verifier la conversion   
    
    l =len(t) #len(KALOUT2)
    
    xi=np.zeros((l,1))
    C=np.zeros((l,2))
    R=np.zeros((l,1))
    r1=np.zeros((l,1))      #calcul de rho1 a partir de z1 et z2
    r2=np.zeros((l,1))      #calcul de rho1 a partir de z3 et z4
    y=np.zeros((l,1))
    x=np.zeros((l,1))
    a1=np.zeros((l,1))      #calcul de alpha1 et alpha2 a partir de x et y
    a2=np.zeros((l,1))
    tht=np.zeros((l,1))     #calcul de theta a partir de z1 et alpha1 (ou z3 et alpha2)

    Ray=np.zeros((l,1))
    yNUM1=np.zeros((l,1))
    yDEN1=np.zeros((l,1))
    yNUM2=np.zeros((l,1))
    yDEN2=np.zeros((l,1))
    

    for i in range (l):
        xi[i]=abs(z[i,2]-z[i,0])
        C[i,0]=(B[1]-A[1]+math.tan(xi[i,0])*(A[0]+B[0]))/(2*math.tan(xi[i,0])) #pourquoi C(1) est il constant?
        C[i,1]=(B[0]-A[0]+math.tan(xi[i,0])*(A[1]+B[1]))/(2*math.tan(xi[i,0]))


# Attention verifier adaptation code        
    for i in range (len(C)):
        Ray[i,0] = math.sqrt( np.dot(C[i],np.transpose(C[i])) )

    #  Ray(:,1)=sqrt((B(1,1)-A(1,1))^2+(B(2,1)-A(2,1))^2)./(2*sin(xi(:,1)));
    #r1(:,1)=sin(pi-z(:,1))./z(:,2);           
    for i in range (len(z)):
        r1[i] = z[i,1]
#    r2(:,1)=sin(pi-z(:,3))./z(:,4);        
    for i in range (l):
        r2[i,0] = AK(z[i,0],z[i,1],z[i,2],B)

    
    for i in range (len(t)):
        yNUM1[i] = (C[i,0]*(r2[i]**2-r1[i]**2)+B[0]*r1[i]**2)

        yDEN1[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        yNUM2[i] = (B[0]*C[i,0]*(C[i,0]-B[0])-B[0]*Ray[i,0]**2+B[0]*C[i,1]**2-C[i,0]*B[1]**2)

        yDEN2[i] =(2*(B[0]*C[i,1]-C[i,0]*B[1]))

        
    y =yNUM1/yDEN1 + yNUM2/yDEN2

    
    for i in range (len(t)):
        x[i,0]=math.sqrt(r1[i,0]**2-y[i,0]**2)
        a1[i,0]=(math.atan2(A[1]-y[i,0],A[0]-x[i,0])-math.pi) % (2*math.pi)
        a2[i,0]=(math.atan2(B[1]-y[i,0],B[0]-x[i,0])-math.pi) % (2*math.pi)
        tht[i,0]=math.pi-z[i,0]+a1[i,0]   

                       
              
                       
"""
    figure(8)
%    title('Trajectoire estimee par l''observateur')
    hold on
    plot(x(:,1),y(:,1),'r')
    plot(XOUT(:,1),XOUT(:,2),'Color',C_trajectory)
    hold off

    legend('Estimated state','Actual Trajectory')
"""                       
                                               #
                                               #                                               
                                               #
                                               #
    ### changement de variable inverse.        #
    ### ------------------------------------ ###    
    

In [ ]:
A1= np.array([[1],[-1],[-0.5],[3],[6],[6]])
B1= np.array([[8],[-2],[-6],[6],[2],[3]])
A1= np.transpose(A1)
B1=np.transpose(B1)
C1=np.array([A1,B1])

A2= np.array([1,-1,-0.5,3,6,6])
B2= np.array([8,-2,-6,6,2,3])
C2=np.array([A2,B2])

#A= np.array([[1,2,3],[-1,4,5],[-0.5,6,7],[3,6,7],[6,4,8],[6,6,8]])
#A= np.transpose(A)
print(A1)
print(B1)
print(C1)

print('\n\n')

print(A2)
print(B2)
print(C2)

print(Z)

In [ ]:
A= np.array([[1,2,3],[-1,4,5],[-0.5,6,7],[3,6,7],[6,4,8],[6,6,8]])
A= np.transpose(A)
A

In [ ]:
len(A)

In [ ]:
A=np.array([[1],[-1],[-0.5],[3]])
B=A[0:3]
B

In [ ]:
zed[0]

In [ ]:
np.diag([1,1,1])

In [ ]:
range(0,10,2)

In [ ]:
a=True
b=True
if a and b:
    print('ok')
else:
    print('not ok')

In [ ]:
S1=np.array([[-1, 0, 1],[5,4,2]])
S2=np.array([[-2, 0, 2],[5,-2,7]])
np.union1d(S1,S2)


In [ ]:
S1=np.array([[-1, 0, 1],[5,4,2]])
[m,n]=S1.shape
print('m =',m,'\n n=',n)

In [ ]:
A=np.array([1,4,7])
B=np.array([1,4,7]) 
a=[1,4,7]
b=[1,4,7] 
print(a,b)
a=np.array(a)
b=np.array(b)
print(A,B)
print(a,b)
print(A==B)
print(a==b)
min(A==B)
min(a==b)

In [ ]:
mes_1

In [ ]:
mes_1.shape

In [ ]:
theta.shape

In [ ]:
test=np.transpose(theta)
test.shape

In [ ]:
A.shape

In [ ]:
np.transpose(A).shape

In [ ]:
print(Angle_temp[106],ya[0], yb[0],A,B)
ii

In [ ]:
newS1=['%.3f' %elem for elem in S1]
%pprint
print(newS1)

In [ ]:
print('ii[52]',ii[52], '\n' ,
't[1651]',t[1651] ,'\n' ,
't[2393]',t[2393],'\n',
ya[1755],'\n',
     Angle_temp[1755])

In [ ]:
temp1

In [ ]:
L1=[0,1,2,3,8,9,10]
t0=L1
t1=L1.copy()
t2=L1[4]
print(L1,t0,t1,t2)
t1[4]+=2
t0[3]=0
print(L1,t0,t1,t2)
t2+=3
print(L1,t0,t1,t2)